## DataMining Assignment 2

In [2]:
import codeUtils as cu

In [3]:
movies = cu.load_data('data/movies.csv')
ratings = cu.load_data('data/ratings.csv')
tags = cu.load_data('data/tags.csv')
links = cu.load_data('data/links.csv')

In [4]:
print("Totoal number of Movies: "+str(len(movies)))
print("Totoal number of Users: "+str(ratings.userId.nunique()))


Totoal number of Movies: 9742
Totoal number of Users: 610


Merging data

### Note: From README.txt
_Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970_

In our case of study this attribute has no importance, so we will get rid of it

In [5]:
cu.drop_columns(tags, ['timestamp'])
cu.drop_columns(ratings, ['timestamp'])

In [6]:
merged_data = cu.merge_data(ratings, movies, 'movieId','inner')

merged_data = cu.merge_data(merged_data, tags, ['userId','movieId'],'left')

cu.drop_na(merged_data)
cu.drop_duplicate(merged_data)

merged_data


,userId,movieId,rating,title,genres,tag
121,336,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar
164,474,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar
193,567,1,3.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,fun
241,289,3,2.5,Grumpier Old Men (1995),Comedy|Romance,moldy
242,289,3,2.5,Grumpier Old Men (1995),Comedy|Romance,old
...,...,...,...,...,...,...
101768,567,170945,3.5,It Comes at Night (2017),Horror|Mystery|Thriller,Suspenseful
101769,567,176419,3.0,Mother! (2017),Drama|Horror|Mystery|Thriller,allegorical
101770,567,176419,3.0,Mother! (2017),Drama|Horror|Mystery|Thriller,uncomfortable
101771,567,176419,3.0,Mother! (2017),Drama|Horror|Mystery|Thriller,unsettling


In [7]:
avreage_rating = cu.calculate_average(merged_data,'movieId','rating')

merged_data = cu.merge_data(merged_data, avreage_rating, 'movieId','inner')

merged_data


,userId,movieId,rating,title,genres,tag,average_rating
0,336,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar,3.833333
1,474,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar,3.833333
2,567,1,3.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,fun,3.833333
3,289,3,2.5,Grumpier Old Men (1995),Comedy|Romance,moldy,2.500000
4,289,3,2.5,Grumpier Old Men (1995),Comedy|Romance,old,2.500000
...,...,...,...,...,...,...,...
3471,567,170945,3.5,It Comes at Night (2017),Horror|Mystery|Thriller,Suspenseful,3.500000
3472,567,176419,3.0,Mother! (2017),Drama|Horror|Mystery|Thriller,allegorical,3.000000
3473,567,176419,3.0,Mother! (2017),Drama|Horror|Mystery|Thriller,uncomfortable,3.000000
3474,567,176419,3.0,Mother! (2017),Drama|Horror|Mystery|Thriller,unsettling,3.000000


In [8]:
new_merged_data = cu.transform_attribute_to_multiple(merged_data, 'genres', '|')

new_merged_data

,userId,movieId,rating,title,genres,tag,average_rating,Adventure,Animation,Children,Comedy,Fantasy,Romance,Mystery,Thriller,Crime,Action,Drama,War,Sci-Fi,Western,Horror,Musical,Film-Noir,IMAX,Documentary,(no genres listed)
0,336,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar,3.833333,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,474,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar,3.833333,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,567,1,3.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,fun,3.833333,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,289,3,2.5,Grumpier Old Men (1995),Comedy|Romance,moldy,2.500000,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,289,3,2.5,Grumpier Old Men (1995),Comedy|Romance,old,2.500000,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3471,567,170945,3.5,It Comes at Night (2017),Horror|Mystery|Thriller,Suspenseful,3.500000,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0
3472,567,176419,3.0,Mother! (2017),Drama|Horror|Mystery|Thriller,allegorical,3.000000,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0
3473,567,176419,3.0,Mother! (2017),Drama|Horror|Mystery|Thriller,uncomfortable,3.000000,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0
3474,567,176419,3.0,Mother! (2017),Drama|Horror|Mystery|Thriller,unsettling,3.000000,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0


In [9]:
cu.drop_columns(new_merged_data, ['genres', 'title', '(no genres listed)'])

In [10]:
new_merged_data = cu.transform_strings_to_numbers(new_merged_data, 'tag')

new_merged_data

,userId,movieId,rating,tag,average_rating,Adventure,Animation,Children,Comedy,Fantasy,Romance,Mystery,Thriller,Crime,Action,Drama,War,Sci-Fi,Western,Horror,Musical,Film-Noir,IMAX,Documentary
0,336,1,4.0,1,3.833333,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,474,1,4.0,1,3.833333,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,567,1,3.5,2,3.833333,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,289,3,2.5,3,2.500000,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,289,3,2.5,4,2.500000,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3471,567,170945,3.5,1540,3.500000,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0
3472,567,176419,3.0,1541,3.000000,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0
3473,567,176419,3.0,1542,3.000000,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0
3474,567,176419,3.0,1543,3.000000,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0


In [13]:
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.cluster import KMeans

# Create a dataframe where each row is a user and each column is a movie. 
user_movie_matrix = new_merged_data.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)

In [14]:
# Use the Apriori algorithm to find frequent itemsets
frequent_itemsets = apriori(user_movie_matrix, min_support=0.05, use_colnames=True)

ValueError: The allowed values for a DataFrame are True, False, 0, 1. Found value 5.0

In [ ]:
# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [ ]:
# Step 3: Clustering
# Create a dataframe where each row is a user and the columns are the average ratings for each genre
user_genre_matrix = new_merged_data.groupby('userId').mean()

# Use K-Means to cluster the users
kmeans = KMeans(n_clusters=5)
kmeans.fit(user_genre_matrix)
clusters = kmeans.predict(user_genre_matrix)

# Add the cluster labels to the user_genre_matrix
user_genre_matrix['cluster'] = clusters

In [ ]:
def denormalize_rating(rating, min_rating, max_rating):
    # Denormalize the rating
    denormalized_rating = rating * (max_rating - min_rating) + min_rating

    # Return the denormalized rating
    return denormalized_rating


In [ ]:
def predict_rating(user_id, movie_id):
    # Get the user's cluster
    user_cluster = user_genre_matrix.loc[user_id, 'cluster']

    # Get the average rating of the movie by the user's cluster
    cluster_avg_rating = merged_data[(merged_data['movieId'] == movie_id) & (merged_data['cluster'] == user_cluster)]['rating'].mean()

    # Get the association rules for the movie
    movie_rules = rules[rules['antecedents'].apply(lambda x: movie_id in x)]

    # If there are no association rules for the movie, return the cluster average rating
    if movie_rules.empty:
        return cluster_avg_rating

    # Calculate the weighted average of the consequent ratings, with the confidence as the weight
    weighted_avg_rating = (movie_rules['consequents'].apply(lambda x: user_movie_matrix.loc[user_id, x]) * movie_rules['confidence']).sum() / movie_rules['confidence'].sum()

    # Return the average of the cluster average rating and the weighted average rating
    return (cluster_avg_rating + weighted_avg_rating) / 2


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
train_data, test_data = train_test_split(new_merged_data, test_size=0.2, random_state=42)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Split the data into training and test sets
train_data, test_data = train_test_split(merged_data, test_size=0.2, random_state=42)

# Assume you have implemented the following function in your codeUtils module
# predict_rating(user_id, movie_id)

# Initialize a list to store the predicted ratings
predicted_ratings = []

# For each user-movie pair in the test set
for index, row in test_data.iterrows():
    # Predict the rating
    predicted_rating = cu.predict_rating(row['userId'], row['movieId'])
    
    # Add the predicted rating to the list of predicted ratings
    predicted_ratings.append(predicted_rating)

# Calculate the mean absolute error between the actual ratings and the predicted ratings
mae = mean_absolute_error(test_data['rating'], predicted_ratings)

print(f'Mean Absolute Error: {mae}')


## Normalization and Standarization

### 1. Normalization

By applying Min-max

In [10]:
normalized_data = cu.normalize_data(new_merged_data, ['rating', 'average_rating', 'tag'])

normalized_data

,userId,movieId,rating,tag,average_rating,Adventure,Animation,Children,Comedy,Fantasy,...,Action,Drama,War,Sci-Fi,Western,Horror,Musical,Film-Noir,IMAX,Documentary
0,336,1,0.777778,0.000000,0.740741,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,474,1,0.777778,0.000000,0.740741,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,567,1,0.666667,0.000649,0.740741,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,289,3,0.444444,0.001297,0.444444,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,289,3,0.444444,0.001946,0.444444,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3471,567,170945,0.666667,0.998054,0.666667,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3472,567,176419,0.555556,0.998703,0.555556,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3473,567,176419,0.555556,0.999351,0.555556,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3474,567,176419,0.555556,1.000000,0.555556,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0


### 2. Standarization

By applying StandardScaler

In [11]:
standrized_data = cu.standardize_data(new_merged_data, ['rating', 'average_rating', 'tag'])

standrized_data

,userId,movieId,rating,tag,average_rating,Adventure,Animation,Children,Comedy,Fantasy,...,Action,Drama,War,Sci-Fi,Western,Horror,Musical,Film-Noir,IMAX,Documentary
0,336,1,-0.019642,-1.339026,-0.224722,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,474,1,-0.019642,-1.339026,-0.224722,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,567,1,-0.603208,-1.336675,-0.224722,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,289,3,-1.770339,-1.334324,-1.857613,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,289,3,-1.770339,-1.331973,-1.857613,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3471,567,170945,-0.603208,2.279183,-0.632945,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3472,567,176419,-1.186773,2.281534,-1.245279,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3473,567,176419,-1.186773,2.283885,-1.245279,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3474,567,176419,-1.186773,2.286236,-1.245279,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0


## Split the data into training and test datasets

In [12]:
nrm_train, nrm_test = cu.split_data(normalized_data)

nrm_train.shape, nrm_test.shape

((2780, 24), (696, 24))

In [13]:
std_train, std_test = cu.split_data(standrized_data)

std_train.shape, std_test.shape

((2780, 24), (696, 24))